In [27]:
from main import gps_preprocessor
from shapely.geometry import Point

### Set Depot Points, Boundary and Buffer here

In [28]:
p1 = Point(19.421231015290946, -99.12768416729142)
p2 = Point(19.420298241433308, -99.12729008610742)
p3 = Point(19.419116629726144, -99.12747041774)
p4 = Point(19.41926477082061, -99.12969297748691)
p5 = Point(19.42117868427646, -99.12944573801215)

points = [p1,p2,p3,p4,p5,p1]
depot, centroid_point = gps_preprocessor.set_depot_boundary(points,buffer = 0)

### Read and Validate GPS Data here

In [29]:
'''
error_factor here is used as 1.5, i.e., 
    relative_error = abs(velocity_gps-velocity_vehicle) / velocity_vehicle 
    =>  ( 5 - 1 ) / 1 > 1.5
    => Calculated GPS speed is way higher than observed speed of the vehicle, i.e. gps walking
    => higher error_factor will allow more GPS walking
'''
error_factor = 1.15

In [30]:
cols = ['UnityLicensePlate', 'RowReferenceTime', 'lat', 'lon','Speed','EngineStatus']
date_col = ['RowReferenceTime']
d_format = 'ISO8601'
# D:\Extra-Projects\MGL-India\Moving Dots
file = 'C:/Users/achad/OneDrive - Microgrid Labs/Desktop/Works (2023)/Current/trip_data_10Units_1Year_V2.csv'
#gps_preprocessor.export_speed_flag_df(file,cols,date_col,d_format,depot,error_factor)

In [31]:
df = gps_preprocessor.read_data(file,cols,date_col,d_format)

In [32]:
df = gps_preprocessor.validate_mandatory_cols(df,'RowReferenceTime','lat','lon','Speed','UnityLicensePlate')

In [33]:
df = gps_preprocessor.add_date_time_month_df(df)

In [34]:
df = df.groupby(by=['vehicleid','date']).apply(gps_preprocessor.add_prev_latlong).reset_index(drop=True)

In [35]:
df = gps_preprocessor.check_veh_within_depot(df,depot)

In [36]:
df = gps_preprocessor.calculate_speed_flag(df, error_factor)

In [37]:
count_df = gps_preprocessor.depot_locator(df)

In [38]:
df = gps_preprocessor.filterby_speed(df)

In [39]:
df = gps_preprocessor.blank_filter(df)

In [40]:
df = gps_preprocessor.add_dh_trips(df,centroid_point)

In [41]:
tolerance = 0.05
linestring_gdf = gps_preprocessor.simplfy_trip(df, tolerance)